In [ ]:
%matplotlib inline
from pprint import pprint
import itertools

import plot
import pandas
import seaborn

from market import Returns_US_1871
from simulate import simulate_withdrawals
import harvesting
from metrics import average, mean, median
from withdrawal import ConstantDollar, EM, VPW, ECM
import montecarlo
import mortality
import metrics

In [ ]:
def sim2(series):
    x = simulate_withdrawals(series, years=40, withdraw=VPW, portfolio=(600000,400000), harvesting=harvesting.N_60_RebalanceHarvesting)
    stocks_prime = [n.portfolio_pre.stocks / n.portfolio_pre.value_n * 100 for n in x]
    portfolio_prime = [n.portfolio_pre.value_n for n in x]   
    return (stocks_prime, portfolio_prime)

def sim3(series):
    x = simulate_withdrawals(series, years=40, withdraw=ConstantDollar, portfolio=(600000,400000), harvesting=harvesting.N_60_RebalanceHarvesting)
    stocks_prime = [n.portfolio_pre.stocks / n.portfolio_pre.value_n * 100 for n in x]
    portfolio_prime = [n.portfolio_pre.value_n for n in x]   
    return (stocks_prime, portfolio_prime)


from decimal import setcontext, ExtendedContext, Decimal
setcontext(ExtendedContext)
import math

r = Returns_US_1871()
total = 0
bondzero = 0
lowest_bonds = []
highest_bonds = []
final_port = []
failures = 0
all_bonds = []
for year in range(1871, 2015):
    y = sim2(r.iter_from(year))
    if 100 in y[0]:
        #print('Bonds hit zero in %d' % year)
        bondzero += 1
    total += 1
    lowest_bonds.append(min(y[0]))
    highest_bonds.append(max(y[0]))
    final_port.append(y[1][-1])

    def f(x):
        if math.isnan(x):
            return Decimal(0)
        else:
            return x

    all_bonds.append(average(map(f, y[0])))
    if y[1][-1] == 0:
        failures += 1
        #print("Failure in", year)
print("Hit zero bonds", bondzero, "out of", total)
print("Failures", failures, "out of", total)
print("Bond average", average(all_bonds))
print("Lowest average", average(lowest_bonds), "Highest average", average(highest_bonds))
print("Final portfolio", average(final_port))
#print(average(filter(lambda x: x != 0, lowest_bonds)))

In [ ]:
def p(bonds, portfolio):
    fig, ax1 = plt.subplots()
    ax1.plot(bonds, 'b')
    ax1.set_ylabel('Bonds Percentage', color='b')
    ax1.set_xlabel('Year of Retirement')
    ax1.set_ylim([0,100])
    for tl in ax1.get_yticklabels():
        tl.set_color('b')

    ax2 = ax1.twinx()
    ax2.plot(portfolio, 'g')
    ax2.set_ylabel('Portfolio Value', color='g')
    ax2.set_ylim([0,2000000])
    for tl in ax2.get_yticklabels():
        tl.set_color('g')
    format_axis_labels_with_commas(ax2.get_yaxis())

    plt.xlabel('Year of Retirement')
    plt.title('Bonds versus Portfolio')
    plt.show()


#r = Returns_US_1871()
#year = 1952
#y = sim3(r.iter_from(year))
#print(average(y[0]))
#plot(y[0], y_label='Stock %', x_label='Year of Retirement', y_lim=(0,105), title='%d Retiree' % year)

In [ ]:
r = Returns_US_1871()
year = 1960
x = sim3(r.iter_from(year))
y = sim2(r.iter_from(year))

x_p = [n if not math.isnan(n) else 0 for n in x[1]]
y_p = [n if not math.isnan(n) else 0 for n in y[1]]

print(len(list(filter(lambda x: x != 0, x_p))))
print(len(list(filter(lambda x: x != 0, y_p))))

lim = None
#lim = (-3,103)
#plot.plot_two(x_p, y_p, y_lim=(0,3000000), title=year, s1_title='Rebalancing', s2_title='Prime Harvesting')
plot.plot_n({'EM' : x_p, 'VPW' : y_p}, 'Year of Retirement', year)

In [ ]:
from harvesting import PrimeHarvesting
from withdrawal import EM
from portfolio import Portfolio

portfolio = Portfolio(392789,284434)
strategy = PrimeHarvesting(portfolio).harvest()
strategy.send(None)
withdrawal_strategy = EM(portfolio, strategy).withdrawals()
annual = []

# Withdrawals happen at the start of the year, so the first time
# we don't have any performance data to send them....
data = withdrawal_strategy.send(None)
annual.append(data)
years = 8
years -= 1

series = Returns_US_1871()

for _, d in zip(range(years), series.iter_from(2008)):
    data = withdrawal_strategy.send(d)
    annual.append(data)

for i in annual: print(i);print()


In [ ]:
survival_fn = mortality.make_mortality(mortality.ANNUITY_2000)

def get_rq(portfolio, age, withdrawal_pct):
    # I can't figure out how to to joint life expectancy so I'll
    # just use female life expectancy for now :/
    life_expectancy = mortality.life_expectancy(None, age)

    stock_pct = round(portfolio.stocks_pct * 100)
    mean = montecarlo.simba_mean[stock_pct]
    stddev = montecarlo.simba_stddev[stock_pct]
    
    return metrics.probability_of_ruin(mean, stddev, life_expectancy, float(withdrawal_pct))

def simulate_risk_quotient(series,
                            portfolio=(600000, 400000),
                            harvesting=PrimeHarvesting,
                            withdraw=VPW,
                            live_until=None):
    portfolio = Portfolio(portfolio[0], portfolio[1])
    strategy = harvesting(portfolio).harvest()
    strategy.send(None)
    withdrawal_strategy = withdraw(portfolio, strategy).withdrawals()
    annual = []

    age = 65
    if not live_until:
        live_until = mortality.gen_age(survival_fn)

    # Withdrawals happen at the start of the year, so the first time
    # we don't have any performance data to send them....
    data = withdrawal_strategy.send(None)
    # Every year after the withdrawal we recalculate our risk quotient.
    if data == None:
        rq = 0
    else:
        rq = get_rq(portfolio, age, data.withdraw_n/data.portfolio_pre.value_n)
        
    annual.append(rq)

    for d in series:
        age += 1
        if age > live_until:
            break

        data = withdrawal_strategy.send(d)
        rq = get_rq(portfolio, age, data.withdraw_n/data.portfolio_pre.value_n)
        annual.append(rq)
    return annual

In [ ]:
def simulate_wer(series,
                portfolio=(600000, 400000),
                harvesting=PrimeHarvesting,
                withdraw=VPW,
                live_until=None):
    portfolio = Portfolio(portfolio[0], portfolio[1])
    strategy = harvesting(portfolio).harvest()
    strategy.send(None)
    withdrawal_strategy = withdraw(portfolio, strategy).withdrawals()
    annual = []

    age = 65
    if not live_until:
        live_until = mortality.gen_age(survival_fn)

    # Withdrawals happen at the start of the year, so the first time
    # we don't have any performance data to send them....
    data = withdrawal_strategy.send(None)
    annual.append(data)

    for d in series:
        age += 1
        if age > live_until:
            break

        data = withdrawal_strategy.send(d)
        annual.append(data)
        
    cfs = [n.withdraw_pct_orig for n in annual]
    returns = [n.returns for n in annual]
    
    # These two sequences are not aligned. The nth element of cfs corresponds to the n+1 element
    # of returns. So we need to shift the returns list around a bit so they become aligned
    # for the WER calculations
    returns.pop(0)
    returns.append(0)

    return (metrics.cew(cfs), metrics.wer(cfs, returns), metrics.hreff(cfs, returns))

In [ ]:
all_frames = {}

series = Returns_US_1871()

for h in [#harvesting.N_10_RebalanceHarvesting,
          harvesting.N_60_RebalanceHarvesting,
          #harvesting.N_70_RebalanceHarvesting,
          #harvesting.N_80_RebalanceHarvesting,
          #harvesting.N_90_RebalanceHarvesting,
          #harvesting.N_100_RebalanceHarvesting,
          #harvesting.AltPrimeHarvesting,
          #harvesting.PrimeHarvesting
    ]:
    #df = pandas.DataFrame(columns=['RQ', 'CEW', 'WER', 'HREFF3'])
    df = pandas.DataFrame(columns=['RQ'])
    all_frames[h.__name__] = df
    for i in range(1871, 1991):
        #(cew, wer, hreff3) = simulate_wer(series.iter_from(i), live_until=90, harvesting=h, withdraw=lambda p, s: VPW(p, s, years_left=50))
        #(cew, wer, hreff3) = simulate_wer(series.iter_from(i), live_until=90, harvesting=h, withdraw=EM)
        #rq = simulate_risk_quotient(series.iter_from(i), live_until=95, harvesting=h, withdraw=lambda p, s: VPW(p, s, years_left=50))
        rq = simulate_risk_quotient(series.iter_from(i), live_until=95, harvesting=h, withdraw=lambda p, s: ConstantDollar(p, s, rate=Decimal(0.035)))
#        if i == 1921:
#            import pdb;pdb.set_trace()
        rq = metrics.average(rq)
        #df.loc[i] = [rq, cew * 1000000, wer, hreff3]
        #df.loc[i] = cew * 1000000
        df.loc[i] = [rq]

In [ ]:
def f(name):
    return (name, round(all_frames[name]['RQ'].median() * 10000) / 100.0)
#print(f(harvesting.N_60_RebalanceHarvesting.__name__))
#print(f(harvesting.N_70_RebalanceHarvesting.__name__))
#print(f(harvesting.N_80_RebalanceHarvesting.__name__))
#print(f(harvesting.N_90_RebalanceHarvesting.__name__))
#print(f(harvesting.N_100_RebalanceHarvesting.__name__))
#print(f(harvesting.PrimeHarvesting.__name__))
#print(f(harvesting.AltPrimeHarvesting.__name__))
plot.plot_n(all_frames, "30-year Average Risk Quotient", "Annual Rebalancing, Constant Dollar Withdrawals", zero_based=True, add_commas=False)
#for f in all_frames:
#    all_frames[f].to_csv('RQ-CSV %s.csv' % f)

In [ ]:
#n = all_frames['60% Stocks']
#print('Number', len(n))
#n[n > 0.05].count()

In [ ]:
year = 1910
#r60 = simulate_risk_quotient(series.iter_from(year), live_until=95, harvesting=harvesting.N_60_RebalanceHarvesting, withdraw=lambda p, s: VPW(p, s, years_left=50))
r60 = simulate_risk_quotient(series.iter_from(year), live_until=95, harvesting=harvesting.N_60_RebalanceHarvesting, withdraw=ConstantDollar)
vpw = simulate_risk_quotient(series.iter_from(year), live_until=95, harvesting=harvesting.N_60_RebalanceHarvesting, withdraw=lambda p, s: VPW(p, s, years_left=35))

#print(metrics.average(r60))
#pprint(metrics.average(ph))
#pprint([(a-b) for (a,b) in zip(r60, ph)])
#pprint(ph)

#plot.plot_n({'Annual Rebalancing': r60, 'Prime Harvesting': ph}, title='%s Retiree' % year, xlabel='Risk Quotient', add_commas=False)
#plot.plot_n({'Annual Rebalancing': r60}, title='%s Retiree' % year, xlabel='Risk Quotient', add_commas=False)

df = pandas.DataFrame({
    '4% Rule' : r60,
    'VPW' : vpw,
})
g = seaborn.relplot(
        data=df,
        kind='line',
        aspect=2,
    )
#g.fig.autofmt_xdate()
g.despine(left=True, bottom=True, offset=20)
g.fig.suptitle(f'Retirement in {year}')